In [3]:
import operator

In [28]:
class Grads(object):
  def __init__(self, grads):
    self.grads = grads
    
  def __add__(self, b):
    grads = {}
    
    return Grads(self.merge(b, operator.add))
  
  def merge(self, b, f):
    a = self.grads
    b = b.grads
    
    merged = {k: a.get(k, b.get(k)) for k in a.keys() ^ b.keys()}
    merged.update({k: f(a[k], b[k]) for k in a.keys() & b.keys()})
    
    return merged
  
  def __getitem__(self, key):
    return self.grads[key]

class Op(object):
  def __add__(self, b):
    return Add(self, b)
  
  def __mul__(self, b):
    return Mul(self, b)
  
class Const(Op):
  def __init__(self, x):
    self.x = x
    self.grads = Grads({})
    
  def eval(self):
    return self.x
  
class Variable(Op):
  def __init__(self, x):
    self.x = x
    self.grads = Grads({self: Const(1)})
    
  def eval(self):
    return self.x

class Add(Op):
  def __init__(self, a, b):
    self.a = a
    self.b = b
    self.grads = a.grads + b.grads
    
  def eval(self):
    return a.eval() + b.eval()
  
class Mul(Op):
  def __init__(self, a, b):
    self.a = a
    self.b = b
    self.grads = a * b.grads + b * a.grads
    
  def eval(self):
    return a.eval() * b.eval()

In [29]:
a = Variable(1)
b = Variable(2)

z = a + b * Const(3)

da, db = z.grads[a], z.grads[b]

print('z = %s' % z.eval())
print('da: %s, db: %s' % (da.eval(), db.eval()))

AttributeError: 'dict' object has no attribute 'grads'